## Custom Preprocessing 

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
class Preprocessing(BaseEstimator, TransformerMixin):
#     # Class Constructor 
#     def __init__(self, feature_names):
#         self._feature_names = feature_names 
    
    # Return self nothing else to do here    
    def fit(self, X, y=None):    
        return self 
    
    # Method that describes what we need this transformer to do
    def transform(self, X, y=None):
        global text_col,user_input
        replacement_chars_list = ['~', '`', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '()', '-', '--', '_', '__', '=', '+', '[', ']', '[]', '{', '}', '{}', '\\', '|', ';', ':', "'", '"', ',', '<', '.', '>', '/', '?', '', ' ', 'na', 'na.', 'n.a.', 'NA', 'NA.', 'N.A.']
        user_input=['fully','@@','JARDIM DA PENHA']
        for i in user_input:
            replacement_chars_list.append(i)
        X.replace(replacement_chars_list,  np.nan, inplace = True)
       
       
        
        # dropping datetime & timedelta[ns] variables from the data
        #X.drop([col for col in X.columns if user_columns_dtypes[col] in ['datetime64', 'datetime64[ns]', 'timedelta64[ns]']], axis = 1, inplace = True)
        X=X.apply(lambda col: pd.to_datetime(col, errors='ignore')if col.dtypes == object  else col,  axis=0)
        date_columns=X.select_dtypes(include=['datetime64[ns, UTC]','datetime64', 'datetime64[ns]', 'timedelta64[ns]'])
        date_col_remove_form_data=list(date_columns.columns)
        X.drop(columns=date_col_remove_form_data,inplace=True)
        date_columns=date_columns.apply(lambda x : pd.to_datetime(x).dt.date)
        date_columns=date_columns.astype('datetime64')
        
        for i in date_col_remove_form_data:
            date_columns[i+'_year']=date_columns[i].dt.year
            date_columns[i+'_month']=date_columns[i].dt.month
            date_columns[i+'_day']=date_columns[i].dt.day
            
        date_columns.drop(columns=date_col_remove_form_data,inplace=True)                                         
        for col in X.columns:        
            pandas_dtype = X[col].dtype
            user_datatypes = user_columns_dtypes
            user_dtype = user_datatypes[col]
        
            if (pandas_dtype in ['int', 'int_', 'int8', 'int16', 'int32', 'int64', 'uint8', 'uint16', 'uint32', 'uint64', 'float', 'float_', 'float16', 'float32', 'float64'] and user_dtype == 'object'): # converting int or float to object
                X[col] = X[col].astype(object)
            elif (pandas_dtype == 'object' and user_dtype == 'int'): # converting object to int
                X[col] = X[col].astype(float).astype('int64')
            elif (pandas_dtype == 'object' and user_dtype == 'float'): # converting object to float
                X[col] = X[col].astype(float)
            elif (pandas_dtype in ['int', 'int_', 'int8', 'int16', 'int32', 'int64', 'uint8', 'uint16', 'uint32'] and user_dtype == 'float'): # converting int to float
                X[col] = X[col].astype(float)
            elif (pandas_dtype in ['float', 'float_', 'float16', 'float32', 'float64'] and user_dtype == 'int'): # converting float to int
#                 X[col] = X[col].astype('int64')
                if X[col].isnull().sum()==0:
                    X[col] = X[col].astype('int64')
            elif (pandas_dtype in ['bool', 'bool_'] and user_dtype == 'object'):
                X[col] = X[col].astype(object) # converting boolean to object
                
        ## -----------------------------text column processing code------------------------------------        
        
        
        #global text_col,user_input
        user_input=['fully','@@','JARDIM DA PENHA']
        stopwords = nltk.corpus.stopwords.words('english')
        for i in user_input:
            stopwords.append(i)

        text_col=[]
        try:
            for i in X.columns.to_list():
                if (X[i].dtype == 'object') and (len(X[i].apply(word_tokenize)[0]) > 3):
                    text_col.append(i)
            print(text_col)
            
        except TypeError:
            
            print('expected string or bytes-like object in dataframe columns')
        text_data_columns=X[text_col]
        #print(X[text_col])
        X.drop(columns=text_col,inplace=True)
        ##NLP---------------------------------------------------
        text_data_columns.replace('[^A-Za-z\s]+', '')
        lemmatizer = WordNetLemmatizer()
        try:
            for i in text_data_columns.columns.to_list():
                text_data_columns[i]=text_data_columns[i].str.lower()
                text_data_columns[i]=text_data_columns[i].apply(word_tokenize)
                text_data_columns[i]=text_data_columns[i].apply(lambda x: [item for item in x if item not in stopwords])
                text_data_columns[i]=text_data_columns[i].apply(lambda x: [lemmatizer.lemmatize(item) for item in x ])
                #print(text_data_columns)
        except AttributeError:
                print('some columns are not string datatype so we cant perform NLP tasks')
        X=pd.concat([date_columns,text_data_columns,X],axis=1)
        return X  

In [ ]:
preprocessing=Preprocessing()
output=preprocessing.transform(data)

## create user_df_dtype_dict dictionary using UI 

In [ ]:
def pandas_dtypes(self):
    global pandas_columns_dtypes
    pandas_columns_dtypes = {}
    for col in self.columns:
        dtype = self[col].dtype
        pandas_columns_dtypes[col] = dtype.name
    return pandas_columns_dtypes


user_df_dtype_dict=pandas_columns_dtypes.copy()

## user input backend -------------
user_df_dtype_dict['SalePrice']='float64'
user_df_dtype_dict['YearBuilt']='float64'
user_df_dtype_dict['YrSold']='float64'
user_df_dtype_dict['OverallQual']='float64'


def user_defined_dtype(user_df_dtype_dict):
    pandas_data= pandas_columns_dtypes.copy()
    pandas_data.update(user_df_dtype_dict)
    user_meta=pd.DataFrame(data=pandas_data.items(),columns=['feature','dtype'])
    user_meta.to_excel('user_meta.xlsx')
    return user_meta

In [ ]:
user_defined_dtype(user_df_dtype_dict)

## Profiling for EDA

In [ ]:
from pandas_profiling import ProfileReport
def AudoEDA(self):
    name =[x for x in globals() if globals()[x] is self][0]
    file=name + '_EDA_analysis.html'
    profile=ProfileReport(self)
    profile.to_file(file)
###   